This file performs a test on the ROI pooling gradient operation. Specifically, given a set of inputs, it will compare the theoretical Jacobian (based on the registered gradient op) and the numerical gradient (calculated by perturbing each input and seeing the difference in the output). We perform this test on a wide range of possible inputs - GPU and CPU, different input shapes and sizes, etc - to ensure that the gradient function performs as expected.

In [1]:
import numpy as np
import PIL
from PIL import Image

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops

import time

import os
home = os.getenv("HOME")

In [ ]:
# Since we've added custom operations, we need to import them. Tensorflow does not automatically add custom ops.
# Adjust the paths below to your tensorflow source folder.

# Import the forward op
roi_pooling_module = tf.load_op_library(
    home + "/packages/tensorflow/bazel-bin/tensorflow/core/user_ops/roi_pooling_op.so")
roi_pooling_op = roi_pooling_module.roi_pooling

# Import the gradient op
roi_pooling_module_grad = tf.load_op_library(
    home + "/packages/tensorflow/bazel-bin/tensorflow/core/user_ops/roi_pooling_op_grad.so")
roi_pooling_op_grad = roi_pooling_module_grad.roi_pooling_grad

In [4]:
# Here we register our gradient op as the gradient function for our ROI pooling op. 
@ops.RegisterGradient("RoiPooling")
def _roi_pooling_grad(op, grad0, grad1):
    # The input gradients are the gradients with respect to the outputs of the pooling layer
    input_grad = grad0
    
    # We need the argmax data to compute the gradient connections
    argmax = op.outputs[1]
    
    # Grab the shape of the inputs to the ROI pooling layer
    input_shape = array_ops.shape(op.inputs[0])
    
    # Compute the gradient
    backprop_grad = roi_pooling_op_grad(input_grad, argmax, input_shape)
    
    # Return the gradient for the feature map, but not for the other inputs
    return [backprop_grad, None, None]

In [5]:
# Set up a Tensorflow session
sess = tf.InteractiveSession()

In [6]:
def test_grad(device, batch_size, num_channels, roi_array, data_type, image_shape, result_shape):
    
    input_shape = (batch_size, num_channels, image_shape[0], image_shape[1])
    roi_array = np.asarray([roi_array for i in range(batch_size)])
    
    # Provision for different ways of making inputs. Currently only testing random.
    if data_type == "linear":
        data_array = np.linspace(1, 0, num=reduce(mul, input_shape, 1)).reshape(input_shape).astype(np.float32)
    else:
        data_array = np.random.random(input_shape).astype(np.float32)
    
    # Choose the device for the test
    with tf.device("/{}:0".format(device)):
        # Set up the inputs
        data_constant = tf.constant(data_array)
        rois_constant = tf.constant(np.asarray(roi_array).astype(np.int32))
        output_shape_tensor_constant = tf.constant(np.asarray(result_shape).astype(np.int32))
        input_shape_constant = array_ops.shape(data_constant)
        
        # Compute the ROI pooling function
        result, argmax = roi_pooling_op(data_constant, rois_constant, output_shape_tensor_constant)
        
        # Compute the theoretical gradient
        #grad_test_op, = tf.gradients(result, data_constant)
        
        # Compute both the theoretic and numeric Jacobians
        theoretic, numeric = tf.test.compute_gradient(data_constant, input_shape, result, 
                                    (input_shape[0],input_shape[1],len(roi_array[0]),result_shape[0],result_shape[1]),
                                   x_init_value=data_array, delta=.00001)
        
        # Check that the arrays are close (within numeric error). If not, print an error message
        if not np.allclose(theoretic, numeric, rtol=.05):
            print("Test Failed: device {}, batch size {}, {} channels, roi_array {}, \
                    {} data, image_shape {}, result_shape {}" \
                  .format(device, batch_size, num_channels, roi_array, data_type, image_shape, result_shape))
            

In [7]:
# Test a bunch of combinations of inputs. Success if no errors are printed.
test_rois = [[0, 0, 10, 10], [2, 1, 7, 9]]
for device in ["cpu", "gpu"]:
    for batch_size in [1,2]:
        for num_channels in [1,2]:
            for num_rois in [1,2]:
                rois = test_rois[:num_rois]
                for image_shape in [(10, 10), (13, 17)]:
                    for result_shape in [(2,2), (1,4)]:
                        # Run the gradient tester on these inputs
                        test_grad(device, batch_size, num_channels, rois, "random", image_shape, result_shape)